In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [12]:
!wget -c -P /tmp/ https://github.com/CSSEGISandData/COVID-19/archive/master.zip

--2020-03-21 19:31:22--  https://github.com/CSSEGISandData/COVID-19/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/CSSEGISandData/COVID-19/zip/master [following]
--2020-03-21 19:31:22--  https://codeload.github.com/CSSEGISandData/COVID-19/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/master.zip’

master.zip              [      <=>           ]  42.84M  3.44MB/s    in 13s     

2020-03-21 19:31:35 (3.41 MB/s) - ‘/tmp/master.zip’ saved [44920474]



In [3]:
!unzip -au /tmp/master.zip -d /tmp/

Archive:  /tmp/master.zip
02be34e5ec0409835f79f61a547b2b42f2c6dfd7
   creating: COVID-19-master/
 extracting: COVID-19-master/.gitignore  [text]  
  inflating: COVID-19-master/README.md  [text]  
   creating: COVID-19-master/archived_data/
  inflating: COVID-19-master/archived_data/README.md  [text]  
   creating: COVID-19-master/archived_data/archived_daily_case_updates/
 extracting: COVID-19-master/archived_data/archived_daily_case_updates/.gitignore  [text]  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-21-2020_2200.csv  [text]  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-22-2020_1200.csv  [text]  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-23-2020_1200.csv  [text]  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-24-2020_0000.csv  [text]  
  inflating: COVID-19-master/archived_data/archived_daily_case_updates/01-24-2020_1200.csv  [text]  
  inflating: COVID-19-master

In [14]:
!wc -l /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/*.csv

    39 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv
    46 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-23-2020.csv
    41 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
    44 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv
    47 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-26-2020.csv
    51 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-27-2020.csv
    52 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
    54 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-29-2020.csv
    58 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-30-2020.csv
    63 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-31-2020.csv
    68 /tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/02-01-2020.csv
    68 /tm

In [15]:
spark = SparkSession.builder.enableHiveSupport().config('spark.sql.repl.eagerEval.enabled', 'true').getOrCreate()

In [17]:
df = spark.read.csv('/tmp/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/*.csv', header=True, inferSchema=True)

In [18]:
df.count()

7313

In [19]:
df.summary()

summary,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
count,4223,7313,7313,7294,6872,6925,4495,4495
mean,null,null,null,624.639292569235,22.502473806752036,244.5965342960289,29.403727522580613,6.430977051167968
stddev,null,null,null,4920.635211307718,220.3237638159897,2706.597705216855,21.80267760767331,85.04737183882683
min,"Montreal, QC",Azerbaijan,1/22/2020 17:00,0,0,0,-41.4545,-157.8584
25%,null,null,null,2.0,0.0,0.0,21.4735,-75.2479
50%,null,null,null,15.0,0.0,1.0,35.4437,12.0
75%,null,null,null,121.0,1.0,12.0,42.9849,90.3563
max,Zhejiang,occupied Palestin...,2020-03-20T23:43:03,996,988,999,72.0,178.065


In [20]:
df = df.select(
    to_timestamp(trim(df['Last Update'])).alias('Last Update'),
    trim(df['Country/Region']).alias('Country/Region'),
    trim(df['Province/State']).alias('Province/State'),
    df.Confirmed.cast('INT'),
    df.Recovered.cast('INT'),
    df.Deaths.cast('INT'),
    df.Latitude.cast('DOUBLE'),
    df.Longitude.cast('DOUBLE'),
    input_file_name()
)

In [22]:
df.summary()

summary,Country/Region,Province/State,Confirmed,Recovered,Deaths,Latitude,Longitude,input_file_name()
count,7313,4223,7294,6925,6872,4495,4495,7313
mean,null,null,624.639292569235,244.5965342960289,22.502473806752036,29.403727522580613,6.430977051167968,null
stddev,null,null,4920.635211307718,2706.597705216855,220.3237638159897,21.80267760767331,85.04737183882683,null
min,Afghanistan,Alabama,0,0,0,-41.4545,-157.8584,file:/tmp/COVID-1...
25%,null,null,2,0,0,21.4735,-75.2479,null
50%,null,null,15,1,0,35.4437,12.0,null
75%,null,null,121,12,1,42.9849,90.3563,null
max,occupied Palestin...,Zhejiang,67800,58382,4032,72.0,178.065,file:/tmp/COVID-1...


In [33]:
df.select(to_date('Last Update').alias('Date'), '*').orderBy(desc_nulls_last('Date')).groupby('Date').sum()

Date,sum(Confirmed),sum(Deaths),sum(Recovered)
2020-03-18,204744,8650,74324
2020-03-17,191145,7839,75441
2020-03-16,173820,7079,71990
2020-03-15,137830,4542,70653
2020-03-14,189820,7769,81023
2020-03-13,82082,3199,63351
2020-03-12,114685,3810,72041
2020-03-11,221994,7874,87629
2020-03-10,114542,4207,60893
2020-03-09,108042,3920,58658


In [34]:
df = df.withColumn('Active', df.Confirmed - (df.Recovered + df.Deaths))

In [35]:
df = df.withColumn('Date', to_date(df['Last Update'])).orderBy(desc_nulls_last('Date'))

In [36]:
df.groupby('Date').sum()

Date,sum(Confirmed),sum(Deaths),sum(Recovered),sum(Active)
2020-03-18,204744,8650,74324,121770
2020-03-17,191145,7839,75441,107865
2020-03-16,173820,7079,71990,94751
2020-03-15,137830,4542,70653,62635
2020-03-14,189820,7769,81023,101028
2020-03-13,82082,3199,63351,15532
2020-03-12,114685,3810,72041,38834
2020-03-11,221994,7874,87629,126491
2020-03-10,114542,4207,60893,49442
2020-03-09,108042,3920,58658,45464


In [37]:
df.groupby('Country/Region').sum()

Country/Region,sum(Confirmed),sum(Deaths),sum(Recovered),sum(Active)
Russia,697,0,105,592
Paraguay,59,0,0,59
North Ireland,1,0,0,1
Senegal,164,0,14,150
Sweden,8767,31,10,8726
Republic of Korea,7513,54,247,7212
Guyana,25,7,0,18
Philippines,1115,108,51,956
Jersey,6,0,0,6
Djibouti,1,0,0,1
